<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank"><< Accessing OSI SAF SST products through the IFREMER FTP server</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_1c_OSI_SAF_SST_data_access_Eumetsat_data_store.ipynb" target="_blank">Accessing data through Eumetsat data store >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sst/HEAD?labpath=1_OSI_SAF_sst_introductory%2F1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1b Accessing Metop SST granule data through IFREMER opensearch queries
### Data used 

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Full resolution Metop-B Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-b" target="_blank">Description</a>| OSI-204-b | 
| Full resolution Metop-C Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-c" target="_blank">Description</a>| OSI-204-c | 


### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF sea surface temperature (SST) products using the IFREMER OpenSearch protocol

### Outline

OSI SAF SST data are available in different distributions. We have explore how to access the low and middle latitude FTP server in the previous notebook. In this notebook, we will explore a way to earch the data by area of interest and period before downloading them. For that, we will search through the **level 2**, full sensor resolution, Metop Near Real Time product, using the <a href="https://opensearch.ifremer.fr/">Ifremer OpenSearch service</a>. OSI SAF is distributing two distinct product for each of the currently operating Metop satellites: <a href="https://osi-saf.eumetsat.int/products/osi-204-b">Metop-B</a> and <a href="https://osi-saf.eumetsat.int/products/osi-204-c">Metop-C</a>. We will see how to search data in each of the products.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Creating the workspace](#section1)
 1. [Setting login information](#section2)
 1. [Defining satellite names and querry parameters](#section3)
 1. [Launch an OpenSearch request](#section4)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [12]:
# library imports
import os               # a library that allows to access to basic operating system commands like making directories
import json             # a library that helps with JSON format files
import requests         # a library that helps retrieve remote data
import xml.dom.minidom  # a library that helps decode xml files
import warnings         # a library that controls python warnings
import datetime         # a library that allows us to use date and time objects
warnings.filterwarnings('ignore')

Next we will create a download directory to store the products we will download in this notebook. <br> 
*Note: this step was normaly already done in previous chapter. If the directory 'product' already exists, the following command will not do anything.*

In [13]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

We will access OSI SAF data from the OSI SAF Lower-Mid-Latitude Centre FTP server in the same way as in the <a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank">previous chapter</a>.

#### Option 1: creating  `.eumetsat_osi_saf_sst_credentials` in our home directory.

If you created the `.eumetsat_osi_saf_sst_credentials` in the <a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank">previous chapter</a>, you just need to execute the command in the following cell : the credentials to access the FTP server will be load from the file you previously created.

Reading credential information

In [14]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sst_credentials')) as json_file:
    credentials = json.load(json_file)

Loading credentials

In [15]:
username = credentials['username'] 
password = credentials['password']

Option 2: provide credentials directly
You can provide your credentials directly as follows; by uncommenting the following lines.

In [16]:
# username = <provided_username>
# password = <provided_password>

You must replace <provided_username> and <provided_password> with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in SST data.

Once you have created an EUMETSAT OSI SAF account, the generic password and credentials to access OSI SAF Lower-Mid-Latitude  server should have been sent to you via email.

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Defining satellite names and querry parameters
[Back to top](#TOC-TOP)

</div>

This first section allows to set the satellite names in order to define a related OpenSearch query

In [17]:
# setting satellite names
SST_MetopB = "avhrr_sst_metop_b-osisaf-l2p-v1.0"
# SST_MetopC = "avhrr_sst_metop_c-osisaf-l2p-v1.0"

Setting the Date, Time, Satellite and Lat° by Lon° box.<br>
We here give you some examples in toggle line. It is up to you to choose.

In [18]:
#acquisition_day = datetime.datetime.now() - datetime.timedelta(days=5)
YYYY = "2025" 
MM = "01" 
DD = "10" 
satellite = SST_MetopB

# # Bretagne + Channel
# lon_min = "-8"
# lon_max = "3"
# lat_min = "45"
# lat_max = "51"

# # Global
# lon_min = "-180"
# lon_max = "180"
# lat_min = "-90"
# lat_max = "90"


# # Gibraltar
# lon_min = "-10"
# lon_max = "-2"
# lat_min = "33"
# lat_max = "38"

# # South Africa
lon_min = "6"
lon_max = "45"
lat_min = "-39"
lat_max = "-14"


# # Japan
# lon_min = "124"
# lon_max = "147"
# lat_min = "25"
# lat_max = "46"

Setting connection information

In [19]:
session = requests.Session()
#username = credentials['username'] 
#password = credentials['password']

<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Launch an OpenSearch request
[Back to top](#TOC-TOP)

</div>

In the upper section we selected the Metop-B/AVHRR dataset Let's then create a specific folder to hold the SST granules (=each file of 3min of measurement by the sensor).<br>
*note: if you choose to download from Metop-C/AVHRR, we advise you to store them in a separate folder name accordingly*

In [20]:
download_dir = os.path.join(os.getcwd(), "products", "L2_AVHRR_Metop-B")
os.makedirs(download_dir, exist_ok=True)

Set the download request

In [21]:
if os.path.exists('searchresult.xml'):
    os.remove('searchresult.xml')
start_time = '07:00:00'
end_time = '14:00:00'
opensearch = requests.get('https://opensearch.ifremer.fr/granules.atom?'\
                          'datasetId=' + satellite + \
                          '&startPage=0&count=1000&'\
                          'timeStart='+YYYY+'-'+MM+'-'+DD+'T' + start_time + 'Z&'\
                          'timeEnd='+YYYY+'-'+MM+'-'+DD+'T' + end_time + 'Z&'\
                          'geoBox='+lon_min+','+lat_min+','+lon_max+','+lat_max, verify=False)
searchresult = open('searchresult.xml','w')
searchresult.write(opensearch.text)
searchresult.close()

Parse granules and download files

In [22]:
# parse granules and download files
xml_resultfile = xml.dom.minidom.parse('searchresult.xml')
link_list = xml_resultfile.getElementsByTagName('link')
granule_number = link_list.length / 3
print("%d granules" % granule_number)
for link in link_list:
    if link.getAttribute('title') == "HTTPS" :
        url = link.getAttribute('href')
        print(url)
        r = requests.get(url, auth=(username, password), verify=False)
        # r.raw.decode_content = True
        filename = url.rsplit('/', 1)[1]
        print(filename)
        with open(os.path.join(os.getcwd(), 'products', 'L2_AVHRR_Metop-B', filename), 'wb') as f:
           f.write(r.content)

8 granules
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2025/010/20250110070103-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_070103-v02.0-fv01.0.nc
20250110070103-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_070103-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2025/010/20250110070403-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_070403-v02.0-fv01.0.nc
20250110070403-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_070403-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2025/010/20250110071003-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_071003-v02.0-fv01.0.nc
20250110071003-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20250110_071003-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2025/010/20250110084003-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop0

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank"><< Accessing OSI SAF SST products through the IFREMER FTP server</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_1c_OSI_SAF_SST_data_access_Eumetsat_data_store.ipynb" target="_blank">Accessing data through Eumetsat data store >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>